In [1]:
%matplotlib widget

In [2]:
import numpy as np

# plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt

#interactive plots
import ipywidgets as widgets
from ipywidgets import VBox, HBox, IntSlider, Checkbox, interactive_output, FloatSlider
from IPython.display import display

import libo
from libo.io import tc3

import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vs


Import movement data

In [3]:
data = tc3.read_db(17519)
ds = tc3.to_xarray(data)

In [4]:
ds[["FB_X","FB_Y","FB_Z"]]

# schweißen
ds.trigSchweissen.dropna("time").plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/tljh/user/envs/libo_stable/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [5]:
locations = ds[["FB_X","FB_Y","FB_Z"]].where(ds.trigSchweissen==1).dropna("time").to_array().data
#locations = np.array([ds.FB_X.dropna("time").data, ds.FB_Y.dropna("time").data, ds.FB_Z.dropna("time").data], float)
locations

array([[  47.188,   47.195,   47.206, ...,   49.738,   49.738,   49.738],
       [1045.312, 1045.315, 1045.31 , ..., 1375.272, 1375.272, 1375.272],
       [-137.386, -137.452, -137.512, ..., -136.894, -136.894, -136.894]])

In [6]:
#plt.plot(locations[0],locations[1])
fig = plt.figure()
ax = fig.gca(projection='3d')
fig.canvas.layout.height="500px"
fig.canvas.layout.width="500px"
ax.plot(locations[0], locations[1], locations[2])
vs.set_axes_equal(ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Define coordinate systems:
- `cs_base` is the root coordinate system
- `cs_ref` is defined in `cs_base` -> coordinate system created from 3 robot positions
- `cs_sp` is defined in `cs_ref` -> specimen coordinate system with x pointing into the direction of the trace

Important note: Measured data is not perfectly orthogonal. We use the cross product to get a z axis which is perfectly orthogonal (in floating point precision) and recalculate a perfectly orthogonal y-axis. There are certainly better ways to adress this problem.

In [7]:
origin = ut.to_float_array([67.311, 1055.137, -132.694])
offset_ox = ut.to_float_array([157.345, 1054.486,-132.949 ])
offset_oy = ut.to_float_array([69.149, 1366.014, -131.962]) 

vec_x = offset_ox - origin
vec_y = offset_oy - origin
vec_z = np.cross(vec_x, vec_y)

cs_ref = tf.LocalCoordinateSystem.construct_from_xz_and_orientation(vec_x=vec_x, vec_z=vec_z, origin=origin)
cs_sp = tf.LocalCoordinateSystem(basis=tf.rotation_matrix_z(np.pi/2),origin=[-20.5, -10 ,-8])

cs_sp_in_base = cs_sp + cs_ref

Transform movement data to specimen coordinate system:

In [9]:
locations_sp = np.matmul(cs_sp_in_base.orientation.transpose(),locations- cs_sp_in_base.location[:,np.newaxis]) 

fig2 = plt.figure()
ax2 = fig2.gca(projection='3d')
fig2.canvas.layout.height="500px"
fig2.canvas.layout.width="500px"
ax2.plot(locations_sp[0], locations_sp[1], locations_sp[2])
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.set_zlabel("z")
vs.set_axes_equal(ax2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
# create points
p_0 = [20, 8]
p_1_1 = [np.tan(3/18 *np.pi)*8, 8]
p_1_2 = [np.tan(2/18 *np.pi)*8, 8]
p_2 = [0, 0]
p_3 = [20, 0]

# create shapes
s_p1_r = geo.Shape().add_line_segments([p_0, p_1_1, p_2, p_3])
s_p2_r = geo.Shape().add_line_segments([p_0, p_1_2, p_2, p_3])
s_p1_l = s_p1_r.reflect([1, 0])
s_p2_l = s_p2_r.reflect([1, 0])

# create profiles
p_1 = geo.Profile([s_p1_l, s_p1_r])
p_2 = geo.Profile([s_p2_l, s_p2_r])

# rasterize and plot profiles
data_p_1 = p_1.rasterize(1)
data_p_2 = p_2.rasterize(1)

fig3 = plt.figure()
plt.plot(data_p_1[0], data_p_1[1], "ro", label="profile 1")
plt.plot(data_p_2[0], data_p_2[1], "bo", label="profile 2")
plt.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Create variable profile, trace and geometry

In [24]:
vp = geo.VariableProfile([p_1, p_2], [0, 1],[geo.linear_profile_interpolation_sbs])
trace = geo.Trace(geo.LinearHorizontalTraceSegment(350))

g_0 = geo.Geometry(vp,trace)

In [30]:
data_g_0 = g_0.rasterize(profile_raster_width=0.5, trace_raster_width=10)

Plot everything together

In [32]:
fig4 = plt.figure()
ax4 = fig4.gca(projection='3d')
fig4.canvas.layout.height="1500px"
fig4.canvas.layout.width="1500px"
ax4.plot(locations_sp[0], locations_sp[1], locations_sp[2], "b")
ax4.plot(data_g_0[0], data_g_0[1], data_g_0[2], "xr")
ax4.set_xlabel("x")
ax4.set_ylabel("y")
ax4.set_zlabel("z")
#vs.set_axes_equal(ax4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'z')